In [55]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import itertools
import pandas as pd
import datetime
from multiprocessing.dummy import Pool  # This is a thread-based Pool
from multiprocessing import cpu_count
pd.set_option('display.max_rows', 1000)

In [61]:

def scraper(search_alph):
    
    df_list = []
    n=len(search_alph)
    time_start = datetime.datetime.now()
    
    for i in range(n):

        print(i)
        time_elapsed = datetime.datetime.now() - time_start
        print('time elapsed: '+str(time_elapsed))
        print('estimated time remaining: '+ str(time_elapsed*(n-i)/20))
        time_start = datetime.datetime.now()
        
        url = 'https://www.genealogy.math.ndsu.nodak.edu/letter.php?letter=' + search_alph[i] + '&fShow=1'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Get table of mathematicians
        table = soup.find('table')
        table_rows = table.find_all('tr')

        # Get IDs
        text = soup.find_all('a', href=True)

        res = []
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text.strip() for tr in td]
            if row:
                res.append(row)

        df_math = pd.DataFrame(res, columns=["Mathematician", "School", "Year"])
        df_math['id'] = [x['href'].split('=')[1] for x in text if 'id.php?id' in x['href']]        
        df_list.append(df_math)
    
    df_final = pd.concat(df_list,axis=0).reset_index(drop=True)
    
    return(df_final) 


In [62]:
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [63]:
def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 


# Parallelize Scraping
chunks = divide_chunks(alphabet,4)
pool = Pool(cpu_count() * 2)
results = pool.map(scraper, chunks)
pool.close()

df_output = pd.concat(results,axis=0).reset_index(drop=True)



000
time elapsed: 0:00:00.000535
estimated time remaining: 0:00:00.000107
0
time elapsed: 0:00:00.001459
estimated time remaining: 0:00:00.000292

0
time elapsed: 0:00:00.000754
estimated time remaining: 0:00:00.000151

time elapsed: 0:00:00.000497
estimated time remaining: 0:00:00.000099
time elapsed: 0:00:00.003995
estimated time remaining: 0:00:00.0007990
0
time elapsed: 0:00:00.000979
estimated time remaining: 0:00:00.000196

time elapsed: 0:00:00.004132
estimated time remaining: 0:00:00.000413
1
time elapsed: 0:00:01.496976
estimated time remaining: 0:00:00.224546
1
time elapsed: 0:00:02.134075
estimated time remaining: 0:00:00.320111
1
time elapsed: 0:00:02.386218
estimated time remaining: 0:00:00.357933
1
time elapsed: 0:00:04.344437
estimated time remaining: 0:00:00.217222
1
time elapsed: 0:00:04.529111
estimated time remaining: 0:00:00.679367
2
time elapsed: 0:00:06.542979
estimated time remaining: 0:00:00.654298
2
time elapsed: 0:00:07.786820
estimated time remaining: 0:00:00

In [64]:
df_output.shape

(248551, 4)

In [65]:
df_output.head()

,Mathematician,School,Year,id
0,"Aab, Michael",Universität Dortmund,1996,29044
1,"Aaberge, Rolf",Universitetet i Oslo,1976,233136
2,"Aaberge, Terje",Université de Genève,1975,160998
3,"Aaboe, Asger",Brown University,1957,13670
4,"Aabrandt, Andreas",Technical University of Denmark,2017,212917


In [68]:
df_output.to_csv('mathematicians.csv', index=False)